In [1]:

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../Data/dontpatronizeme_pcl.tsv', sep = '\t', names=['id','info','country', 'text','label'] )

In [3]:
df = df.dropna(inplace = False)

df = df.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10468 entries, 0 to 10467
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       10468 non-null  object
 1   info     10468 non-null  object
 2   country  10468 non-null  object
 3   text     10468 non-null  object
 4   label    10468 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 409.0+ KB


In [4]:
df_final = df[['text','label']]
df_final
df_final['label_']= [ 0 if (y == 1 or y == 0) else 1 for y in df_final['label']]
df_final
df_final.drop('label', axis = 1, inplace= True)
df_final.head()

C:\Users\PREETA~1\AppData\Local\Temp/ipykernel_13184/1084671562.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['label_']= [ 0 if (y == 1 or y == 0) else 1 for y in df_final['label']]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,text,label_
0,"We 're living in times of absolute insanity , ...",0
1,"In Libya today , there are countless number of...",0
2,White House press secretary Sean Spicer said t...,0
3,Council customers only signs would be displaye...,0
4,""" Just like we received migrants fleeing El Sa...",0


In [5]:
X = df_final.drop('label_', axis =1) #getting independent variable

In [7]:
y = df_final['label_']

In [8]:
y.value_counts()

0    9475
1     993
Name: label_, dtype: int64

In [9]:
X.shape

(10468, 1)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.7.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [15]:
voc_size = 5000

In [16]:
messages = X.copy()

In [18]:
messages['text'][1]

"In Libya today , there are countless number of Ghanaian and Nigerian immigrants . These are the two countries with key macroeconomic challenges including unemployment . Let 's tackle this issue from the root and not the fruit . Thank you"

In [19]:
messages.reset_index(inplace= True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Preetam
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus =[]
from tqdm import tqdm 
for i in tqdm(range(0,10468)):
    
    review = re.sub('[^a-zA-Z]',' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words ('english')]
    review =' '.join(review)
    corpus.append(review)
    

100%|███████████████████████████████████████████████████████████████████████████| 10468/10468 [01:07<00:00, 154.44it/s]


In [25]:
corpus

['live time absolut insan pretti sure peopl awar wake everi day check news seem carri feel panic dread action hero probabl face tri decid whether cut blue green wire tick bomb except bomb instruct long ago burn fire immin catastroph seem likeliest outcom hard stay edg long though natur peopl becom inur constant chao slump malais hopeless pessim',
 'libya today countless number ghanaian nigerian immigr two countri key macroeconom challeng includ unemploy let tackl issu root fruit thank',
 'white hous press secretari sean spicer said focu immigr action would illeg immigr also otherwis violat law',
 'council custom sign would display two space would reserv disabl person would five p space eight p one',
 'like receiv migrant flee el salvador guatemala like seen thousand econom refuge start see mexican flee violenc ju rez part mexico say shelter director ruben garcia',
 'bring high blood sugar level insulin need taken type requir insulin meal time take correct dose insulin order lower blood

In [24]:
onehot_repr = [one_hot(words, voc_size)for words in corpus]
onehot_repr

[[738,
  2110,
  4735,
  3492,
  3491,
  941,
  2863,
  4089,
  4623,
  4802,
  3484,
  476,
  615,
  843,
  4343,
  2620,
  855,
  4806,
  941,
  773,
  3079,
  4250,
  1839,
  2744,
  44,
  2491,
  1009,
  1449,
  3213,
  4147,
  3448,
  4373,
  3448,
  4041,
  3066,
  3959,
  4908,
  2751,
  1118,
  22,
  843,
  3101,
  2015,
  3357,
  3401,
  3269,
  3066,
  2988,
  977,
  2863,
  301,
  4673,
  4614,
  4834,
  4712,
  1784,
  1616,
  4509],
 [2609,
  2349,
  933,
  2722,
  4446,
  3394,
  2987,
  1208,
  2581,
  3999,
  1676,
  998,
  1517,
  4255,
  2282,
  4185,
  4183,
  4095,
  4066,
  1054],
 [4042,
  4668,
  2823,
  522,
  4107,
  1518,
  2736,
  618,
  2987,
  941,
  2370,
  44,
  2987,
  4519,
  867,
  1847,
  3549],
 [2487,
  4064,
  640,
  2370,
  4691,
  1208,
  1666,
  2370,
  3351,
  648,
  1087,
  2370,
  3475,
  2771,
  1666,
  4136,
  2771,
  3404],
 [4970,
  1852,
  2801,
  4351,
  2480,
  1190,
  2397,
  4970,
  3784,
  165,
  1092,
  1806,
  2253,
  4214,
  4459

In [33]:
sent_length = 60
embedded_docs= pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0  738 ... 1784 1616 4509]
 [   0    0    0 ... 4095 4066 1054]
 [   0    0    0 ...  867 1847 3549]
 ...
 [   0    0    0 ... 2621 1324 1487]
 [   0    0    0 ... 1602 2426 1346]
 [   0    0    0 ... 4729 1864 3484]]


In [34]:
embedded_docs[0]

array([   0,    0,  738, 2110, 4735, 3492, 3491,  941, 2863, 4089, 4623,
       4802, 3484,  476,  615,  843, 4343, 2620,  855, 4806,  941,  773,
       3079, 4250, 1839, 2744,   44, 2491, 1009, 1449, 3213, 4147, 3448,
       4373, 3448, 4041, 3066, 3959, 4908, 2751, 1118,   22,  843, 3101,
       2015, 3357, 3401, 3269, 3066, 2988,  977, 2863,  301, 4673, 4614,
       4834, 4712, 1784, 1616, 4509])

In [35]:
embedding_vector_features = 50
model =Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation ='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer ='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
embedding_vector_features = 50
model1 =Sequential()
model1.add(Embedding(voc_size,embedding_vector_features, input_length = sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation ='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer ='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 60, 50)            250000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              120800    
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 371,001
Trainable params: 371,001
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
len(embedded_docs), y.shape

(10468, (10468,))

In [38]:
X_final= np.array(embedded_docs)
y_final=np.array(y)

In [39]:
X_final.shape, y_final.shape

((10468, 60), (10468,))

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.20, random_state= 0)

In [62]:
y_train.shape, y_test.shape

((8374,), (2094,))

In [43]:
model.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=10, batch_size =64)

Epoch 1/10
131/131 [==============================] - 8s 48ms/step - loss: 0.3355 - accuracy: 0.9027 - val_loss: 0.2944 - val_accuracy: 0.9031
Epoch 2/10
131/131 [==============================] - 6s 44ms/step - loss: 0.2209 - accuracy: 0.9140 - val_loss: 0.2717 - val_accuracy: 0.9069
Epoch 3/10
131/131 [==============================] - 6s 44ms/step - loss: 0.1496 - accuracy: 0.9421 - val_loss: 0.3322 - val_accuracy: 0.9031
Epoch 4/10
131/131 [==============================] - 6s 44ms/step - loss: 0.1012 - accuracy: 0.9636 - val_loss: 0.3830 - val_accuracy: 0.8906
Epoch 5/10
131/131 [==============================] - 6s 45ms/step - loss: 0.0626 - accuracy: 0.9793 - val_loss: 0.4148 - val_accuracy: 0.8883
Epoch 6/10
131/131 [==============================] - 6s 47ms/step - loss: 0.0424 - accuracy: 0.9876 - val_loss: 0.5244 - val_accuracy: 0.8849
Epoch 7/10
131/131 [==============================] - 6s 45ms/step - loss: 0.0208 - accuracy: 0.9940 - val_loss: 0.5973 - val_accuracy: 0.8844

In [44]:
model1.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=10, batch_size =64)

Epoch 1/10
131/131 [==============================] - 12s 80ms/step - loss: 0.3319 - accuracy: 0.9016 - val_loss: 0.2885 - val_accuracy: 0.9026
Epoch 2/10
131/131 [==============================] - 10s 76ms/step - loss: 0.2223 - accuracy: 0.9119 - val_loss: 0.2768 - val_accuracy: 0.9040
Epoch 3/10
131/131 [==============================] - 10s 75ms/step - loss: 0.1560 - accuracy: 0.9366 - val_loss: 0.3320 - val_accuracy: 0.9040
Epoch 4/10
131/131 [==============================] - 10s 75ms/step - loss: 0.0995 - accuracy: 0.9615 - val_loss: 0.3919 - val_accuracy: 0.8897
Epoch 5/10
131/131 [==============================] - 10s 74ms/step - loss: 0.0605 - accuracy: 0.9803 - val_loss: 0.4124 - val_accuracy: 0.8801
Epoch 6/10
131/131 [==============================] - 10s 77ms/step - loss: 0.0335 - accuracy: 0.9901 - val_loss: 0.5424 - val_accuracy: 0.8749
Epoch 7/10
131/131 [==============================] - 11s 81ms/step - loss: 0.0132 - accuracy: 0.9953 - val_loss: 0.6512 - val_accuracy:

In [70]:

y_pred = model1.predict(X_test)
classes_x=np.argmax(y_pred, axis =1)

In [68]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1890    0]
 [ 204    0]]


In [69]:
accuracy_score(y_test, classes_x)

0.9025787965616046

In [66]:
from sklearn.metrics import classification_report
print(classification_report(y_test, classes_x))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1890
           1       0.00      0.00      0.00       204

    accuracy                           0.90      2094
   macro avg       0.45      0.50      0.47      2094
weighted avg       0.81      0.90      0.86      2094



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
